In [1]:
%run preprocess_functions.ipynb
%run data_augmentation.ipynb

In [92]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import cv2
import torch
import matplotlib.pyplot as plt
from torch import optim
from torch import nn
import pickle
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score, classification_report, accuracy_score, precision_recall_curve, auc, roc_auc_score, roc_curve, multilabel_confusion_matrix

In [6]:
DIRECTORY_PATH = 'one_eye_images'
AUGMENTED_DIRECTORY_PATH ="original_and_augmented_images"

In [64]:
def save_model(train_dataloader, validation_dataloader, test_dataloader, optimizer, model, train_losses, val_losses, epoch):
    with open(f'train_losses_{epoch}.pkl', 'wb') as file:
        pickle.dump(train_losses, file)
    with open(f'val_losses_{epoch}.pkl', 'wb') as file:
        pickle.dump(val_losses, file)

    torch.save(model.state_dict(), f'vgg_model_{epoch}.pth')
    torch.save(train_dataloader, f'train_dataloader_{epoch}.pth')
    torch.save(validation_dataloader, f'validation_dataloader_{epoch}.pth')
    torch.save(test_dataloader, f'test_dataloader_{epoch}.pth')
    torch.save(optimizer.state_dict(), f'optimizer_{epoch}.pth')

# model for one eye input

Dataset class

In [93]:
class CustomDataset(Dataset):
    def __init__(self, dataset_X, dataset_y, image_directory_path, transfrom=None, include_preprocess_function=False, mode='train'):
        self.X = dataset_X
        self.y = dataset_y
        self.image_directory_path = image_directory_path
        self.preprosecc_status = include_preprocess_function
        
        
        if mode == 'train':
            if self.preprosecc_status:
                self.transform = transfrom or transforms.Compose([
                    # transforms.RandomHorizontalFlip(),
                    # transforms.RandomRotation(degrees=90),
                    # transforms.Lambda(lambda img: Image.fromarray(preprocess(np.array(img)))),
                    transforms.RandomHorizontalFlip(),
                    transforms.RandomVerticalFlip(),
                    transforms.ColorJitter(brightness=0.5, contrast=0.2, saturation=0.2, hue=0),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) #values from image net, better for pretrained models, can be changed to dataset values
                ])
            else:
                self.transform = transfrom or transforms.Compose([
                    # transforms.Lambda(lambda img: Image.fromarray(crop_image_to_circle(np.array(img)))), #1111111
                    transforms.Resize((224, 224)),
                    transforms.RandomHorizontalFlip(),
                    transforms.RandomVerticalFlip(),
                    transforms.ColorJitter(brightness=0.5, contrast=0.2, saturation=0.2, hue=0),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) #values from image net, better for pretrained models, can be changed to dataset values
                ])
        else:
            if self.preprosecc_status:
                self.transform = transfrom or transforms.Compose([
                    # transforms.Lambda(lambda img: Image.fromarray(preprocess(np.array(img)))),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                ])
            else:
                self.transform = transfrom or transforms.Compose([
                    # transforms.Lambda(lambda img: Image.fromarray(crop_image_to_circle(np.array(img)))),
                    transforms.Resize((224, 224)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) #values from image net, better for pretrained models, can be changed to dataset values
                    ])
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        eye_image = cv2.imread(os.path.join(self.image_directory_path, self.X.iloc[index]['image_id']))
        if self.preprosecc_status:
            preprocessed_image = Image.fromarray(preprocess((eye_image)))
        else:
            preprocessed_image = Image.fromarray(crop_image_to_circle(eye_image))
            
        diagnosis = self.y.iloc[index][['diabetic_retinopathy', 'amd', 'hypertensive_retinopathy', 
                                                'normal_eye', 'glaucoma', 'cataract']].to_numpy(dtype=np.float32)
        age = self.X.iloc[index]['patient_age']
        sex = self.X.iloc[index]['patient_sex']
        if self.transform:
           preprocessed_image = self.transform(preprocessed_image)
        
        data = {"eye_image": preprocessed_image,
                "diagnosis": torch.tensor(diagnosis, dtype=torch.float32),
                "metadata": torch.tensor(np.array([age, sex]), dtype=torch.float32)}
        
        return data['eye_image'], data['metadata'], data['diagnosis']

Model class

1. vgg 

In [97]:
class NetworkVGG(nn.Module):
    def __init__(self):
        super(NetworkVGG, self).__init__()
        
        self.eye_input = models.vgg16(weights='VGG16_Weights.DEFAULT').requires_grad_(False)
        self.eye_input = nn.Sequential(*list(self.eye_input.children())[:-1]) #removing the classifier layer 
 
        self.metadata_input = nn.Sequential(
            nn.Linear(2, 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, 64),
            nn.ReLU(inplace=True)
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(25088 + 64, 4096), #4096
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(512, 6),
            # nn.Softmax(dim=1)
        )
        
    def forward(self, eye_image, metadata): 
        vgg = self.eye_input(eye_image)
        vgg = torch.flatten(vgg, 1)
        meta = self.metadata_input(metadata)
        concatenated = torch.cat((vgg, meta), dim=1)
        result = self.classifier(concatenated)
        return result

train function

In [98]:
def train(dataloader, model, loss_function, optimizer):
    size = len(dataloader.dataset)
    running_loss = 0.
    all_predictions = []
    all_labels = []
    model.train()
    for batch, (image, metadata, diagnosis) in enumerate(dataloader): #(image, metadata, diagnosis)

        prediction = model(image, metadata) 
        # print("prediction", prediction[0])
        # print("diagnosis", diagnosis[0])
        loss = loss_function(prediction, diagnosis)
        running_loss += loss.item()
        
        softmax = nn.Softmax(dim=1)
        probabilities = softmax(prediction)
        preds = torch.argmax(probabilities, dim=1)
        all_predictions.extend(preds.cpu().numpy())
        all_labels.extend(torch.argmax(diagnosis, dim=1).cpu().numpy())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 2 == 0:
            loss, current = loss.item(), (batch + 1) * len(diagnosis)
            print(f"loss: {loss:>7f}, [{current:>5d}/{size:>5d}]")
    f1 = f1_score(all_labels, all_predictions, average='macro')
    training_loss = running_loss / len(dataloader)
    print(f"training_loss: {training_loss:>7f}, f1_score: {f1:>7f}")
    return training_loss, f1

In [99]:
def validation(dataloader, model, loss_function):
    val_loss = 0.
    model.eval()
    all_predictions = []
    all_labels = []
    
    with torch.no_grad():
        for (image, metadata, diagnosis) in dataloader: #(image, metadata, diagnosis)
            prediction = model(image, metadata) #, metadata
            batch_val_loss = loss_function(prediction, diagnosis)
            val_loss += batch_val_loss
            softmax = nn.Softmax(dim=1)
            probabilities = softmax(prediction)
            preds = torch.argmax(probabilities, dim=1)
            all_predictions.extend(preds.cpu().numpy())
            all_labels.extend(torch.argmax(diagnosis, dim=1).cpu().numpy())
    validation_loss = val_loss / len(dataloader)
    f1 = f1_score(all_labels, all_predictions, average='macro')
    classification_overall = classification_report(all_labels, all_predictions)
    print(f"validation_loss: {validation_loss:>7f}, f1_score: {f1:>7f}")
    return validation_loss, f1, classification_overall

In [44]:
# def test(dataloader, model, loss_function):
#     num_batches = len(dataloader)
#     test_loss = 0.
#     model.eval()
#     with torch.no_grad():
#         for (image, metadata, diagnosis) in dataloader:
#             prediction = model(image, metadata)
#             batch_test_loss = loss_function(prediction, diagnosis)
#             test_loss += batch_test_loss.item()
#             print(test_loss)
#     return test_loss / num_batches

In [100]:
data = pd.read_csv("final_one_eye_dataset.csv")
classes = ['diabetic_retinopathy', 'amd', 'hypertensive_retinopathy', 'normal_eye', 'glaucoma', 'cataract']
X = data[['image_id', 'patient_age', 'patient_sex']]
y = data[classes]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)


In [90]:
data[classes].value_counts()

diabetic_retinopathy  amd  hypertensive_retinopathy  normal_eye  glaucoma  cataract
0                     0    0                         1           0         0           3094
1                     0    0                         0           0         0           2376
0                     1    0                         0           0         0            468
                      0    0                         0           0         1            292
                           1                         0           0         0            277
                           0                         0           1         0            268
Name: count, dtype: int64

In [89]:
y_train[classes].value_counts()

diabetic_retinopathy  amd  hypertensive_retinopathy  normal_eye  glaucoma  cataract
0                     0    0                         1           0         0           2498
1                     0    0                         0           0         0           1879
0                     1    0                         0           0         0            382
                      0    0                         0           0         1            233
                           1                         0           0         0            215
                           0                         0           1         0            213
Name: count, dtype: int64

datasets and dataloaders

In [101]:
age_scaler = StandardScaler()
with open(f'age_scaler.pkl', 'wb') as file:
    pickle.dump(age_scaler, file)

X_train['patient_age'] = age_scaler.fit_transform(X_train[['patient_age']])
X_val['patient_age'] = age_scaler.transform(X_val[['patient_age']])
X_test['patient_age'] = age_scaler.transform(X_test[['patient_age']])

train_dataset = CustomDataset(X_train, y_train, DIRECTORY_PATH, include_preprocess_function=False)
validation_dataset = CustomDataset(X_val, y_val, DIRECTORY_PATH, mode='test', include_preprocess_function=False)
test_dataset = CustomDataset(X_test, y_test, DIRECTORY_PATH, mode='test', include_preprocess_function=False)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)


    # Compute class weights using class indices
y_train_indices = np.argmax(y_train, axis=1)
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_indices), y=y_train_indices)
class_weights = torch.tensor(class_weights, dtype=torch.float32)

In [ ]:
epochs_vgg16 = 3
learning_rate_vgg = 0.0001
model_vgg = NetworkVGG()
loss_function = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model_vgg.parameters(), lr=learning_rate_vgg)
train_losses = []
val_losses = []
f1_scores_train = []
f1_scores_val = []
reports = []

for t in range(epochs_vgg16):
    print(f"Epoch {t+1}\n-------------------------------")
    epoch_train_loss, f1_score_train = train(train_dataloader, model_vgg, loss_function, optimizer)
    epoch_val_loss, f1_score_val, report = validation(validation_dataloader, model_vgg, loss_function)
    train_losses.append(epoch_train_loss)
    val_losses.append(epoch_val_loss)
    f1_scores_val.append(f1_score_val)
    f1_scores_train.append(f1_score_train)
    save_model(train_dataloader, validation_dataloader, test_dataloader, optimizer, model_vgg, train_losses, val_losses, epoch=t)
print("Done!")

Epoch 1
-------------------------------
loss: 1.795694, [   64/ 5420]
loss: 1.299675, [  192/ 5420]
loss: 2.107519, [  320/ 5420]
loss: 1.829301, [  448/ 5420]
loss: 1.291313, [  576/ 5420]
loss: 1.240274, [  704/ 5420]
loss: 2.078921, [  832/ 5420]
loss: 1.919834, [  960/ 5420]
loss: 1.625710, [ 1088/ 5420]
loss: 1.623205, [ 1216/ 5420]
loss: 1.536857, [ 1344/ 5420]
loss: 1.376989, [ 1472/ 5420]
loss: 1.211249, [ 1600/ 5420]
loss: 1.220406, [ 1728/ 5420]
loss: 1.630548, [ 1856/ 5420]
loss: 1.526355, [ 1984/ 5420]
loss: 1.502885, [ 2112/ 5420]
loss: 0.927267, [ 2240/ 5420]
loss: 1.177265, [ 2368/ 5420]
loss: 1.187661, [ 2496/ 5420]
loss: 1.630650, [ 2624/ 5420]
loss: 1.450040, [ 2752/ 5420]
loss: 1.004056, [ 2880/ 5420]
loss: 0.854994, [ 3008/ 5420]
loss: 1.627032, [ 3136/ 5420]
loss: 1.362912, [ 3264/ 5420]
loss: 1.457378, [ 3392/ 5420]
loss: 1.115484, [ 3520/ 5420]
loss: 0.962503, [ 3648/ 5420]
loss: 1.178239, [ 3776/ 5420]
loss: 1.669788, [ 3904/ 5420]
loss: 1.528483, [ 4032/ 5420]


In [105]:
epoch = 7

for t in range(epoch):
    print(f"Epoch {t+4}\n-------------------------------")
    epoch_train_loss, f1_score_train = train(train_dataloader, model_vgg, loss_function, optimizer)
    epoch_val_loss, f1_score_val, report = validation(validation_dataloader, model_vgg, loss_function)
    train_losses.append(epoch_train_loss)
    val_losses.append(epoch_val_loss)
    f1_scores_val.append(f1_score_val)
    f1_scores_train.append(f1_score_train)
    save_model(train_dataloader, validation_dataloader, test_dataloader, optimizer, model_vgg, train_losses, val_losses, epoch=t+3)
print("Done!")

Epoch 4
-------------------------------
loss: 1.079822, [   64/ 5420]
loss: 0.887209, [  192/ 5420]
loss: 1.354452, [  320/ 5420]
loss: 0.830304, [  448/ 5420]
loss: 1.147152, [  576/ 5420]
loss: 0.927999, [  704/ 5420]
loss: 0.844683, [  832/ 5420]
loss: 1.008545, [  960/ 5420]
loss: 0.849531, [ 1088/ 5420]
loss: 0.935016, [ 1216/ 5420]
loss: 1.251576, [ 1344/ 5420]
loss: 0.883294, [ 1472/ 5420]
loss: 1.018114, [ 1600/ 5420]
loss: 1.286286, [ 1728/ 5420]
loss: 1.032989, [ 1856/ 5420]
loss: 0.860483, [ 1984/ 5420]
loss: 0.822671, [ 2112/ 5420]
loss: 1.198042, [ 2240/ 5420]
loss: 1.104904, [ 2368/ 5420]
loss: 1.026367, [ 2496/ 5420]
loss: 0.729244, [ 2624/ 5420]
loss: 0.911466, [ 2752/ 5420]
loss: 1.277391, [ 2880/ 5420]
loss: 1.045870, [ 3008/ 5420]
loss: 0.978258, [ 3136/ 5420]
loss: 0.848554, [ 3264/ 5420]
loss: 0.660294, [ 3392/ 5420]
loss: 0.798555, [ 3520/ 5420]
loss: 1.158545, [ 3648/ 5420]
loss: 0.966056, [ 3776/ 5420]
loss: 1.075726, [ 3904/ 5420]
loss: 1.234120, [ 4032/ 5420]


In [107]:
epoch = 10

for t in range(epoch):
    print(f"Epoch {t+11}\n-------------------------------")
    epoch_train_loss, f1_score_train = train(train_dataloader, model_vgg, loss_function, optimizer)
    epoch_val_loss, f1_score_val, report = validation(validation_dataloader, model_vgg, loss_function)
    train_losses.append(epoch_train_loss)
    val_losses.append(epoch_val_loss)
    f1_scores_val.append(f1_score_val)
    f1_scores_train.append(f1_score_train)
    save_model(train_dataloader, validation_dataloader, test_dataloader, optimizer, model_vgg, train_losses, val_losses, epoch=t+10)
print("Done!")

Epoch 11
-------------------------------
loss: 1.025759, [   64/ 5420]
loss: 0.920818, [  192/ 5420]
loss: 0.875306, [  320/ 5420]
loss: 0.900582, [  448/ 5420]
loss: 0.802534, [  576/ 5420]
loss: 0.672518, [  704/ 5420]
loss: 0.555936, [  832/ 5420]
loss: 0.929858, [  960/ 5420]
loss: 0.983366, [ 1088/ 5420]
loss: 0.955932, [ 1216/ 5420]
loss: 0.577956, [ 1344/ 5420]
loss: 0.773027, [ 1472/ 5420]
loss: 0.871084, [ 1600/ 5420]
loss: 0.760660, [ 1728/ 5420]
loss: 0.902951, [ 1856/ 5420]
loss: 0.779482, [ 1984/ 5420]
loss: 1.158786, [ 2112/ 5420]
loss: 0.615348, [ 2240/ 5420]
loss: 0.722344, [ 2368/ 5420]
loss: 0.718271, [ 2496/ 5420]
loss: 0.893693, [ 2624/ 5420]
loss: 0.571629, [ 2752/ 5420]
loss: 0.880167, [ 2880/ 5420]
loss: 0.454778, [ 3008/ 5420]
loss: 0.721733, [ 3136/ 5420]
loss: 0.687688, [ 3264/ 5420]
loss: 0.724421, [ 3392/ 5420]
loss: 0.945482, [ 3520/ 5420]
loss: 0.717794, [ 3648/ 5420]
loss: 0.683568, [ 3776/ 5420]
loss: 0.964663, [ 3904/ 5420]
loss: 0.566168, [ 4032/ 5420]

In [114]:
epoch = 10

for t in range(epoch):
    print(f"Epoch {t+21}\n-------------------------------")
    epoch_train_loss, f1_score_train = train(train_dataloader, model_vgg, loss_function, optimizer)
    epoch_val_loss, f1_score_val, report = validation(validation_dataloader, model_vgg, loss_function)
    train_losses.append(epoch_train_loss)
    val_losses.append(epoch_val_loss)
    f1_scores_val.append(f1_score_val)
    f1_scores_train.append(f1_score_train)
    save_model(train_dataloader, validation_dataloader, test_dataloader, optimizer, model_vgg, train_losses, val_losses, epoch=t+20)
print("Done!")

Epoch 21
-------------------------------
loss: 0.559478, [   64/ 5420]
loss: 0.412118, [  192/ 5420]
loss: 0.611295, [  320/ 5420]
loss: 0.435360, [  448/ 5420]
loss: 0.883408, [  576/ 5420]
loss: 0.761217, [  704/ 5420]
loss: 0.454413, [  832/ 5420]
loss: 0.410088, [  960/ 5420]
loss: 0.500075, [ 1088/ 5420]
loss: 0.480168, [ 1216/ 5420]
loss: 0.438511, [ 1344/ 5420]
loss: 0.817518, [ 1472/ 5420]
loss: 0.544629, [ 1600/ 5420]
loss: 0.620723, [ 1728/ 5420]
loss: 0.489719, [ 1856/ 5420]
loss: 0.718344, [ 1984/ 5420]
loss: 0.506929, [ 2112/ 5420]
loss: 0.665701, [ 2240/ 5420]
loss: 0.762601, [ 2368/ 5420]
loss: 0.671271, [ 2496/ 5420]
loss: 0.708611, [ 2624/ 5420]
loss: 0.632877, [ 2752/ 5420]
loss: 0.767061, [ 2880/ 5420]
loss: 0.601696, [ 3008/ 5420]
loss: 0.557894, [ 3136/ 5420]
loss: 0.736177, [ 3264/ 5420]
loss: 0.499508, [ 3392/ 5420]
loss: 0.995557, [ 3520/ 5420]
loss: 0.650416, [ 3648/ 5420]
loss: 0.728390, [ 3776/ 5420]
loss: 0.850621, [ 3904/ 5420]
loss: 0.531427, [ 4032/ 5420]

In [ ]:
data = pd.read_csv("final_one_eye_dataset.csv")
classes = ['diabetic_retinopathy', 'amd', 'hypertensive_retinopathy', 'normal_eye', 'glaucoma', 'cataract']
X = data[['image_id', 'patient_age', 'patient_sex']]
y = data[classes]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

age_scaler = StandardScaler()
with open(f'age_scaler.pkl', 'wb') as file:
    pickle.dump(age_scaler, file)

X_train['patient_age'] = age_scaler.fit_transform(X_train[['patient_age']])
X_val['patient_age'] = age_scaler.transform(X_val[['patient_age']])
X_test['patient_age'] = age_scaler.transform(X_test[['patient_age']])

train_dataset = CustomDataset(X_train, y_train, DIRECTORY_PATH, include_preprocess_function=False)
validation_dataset = CustomDataset(X_val, y_val, DIRECTORY_PATH, mode='test', include_preprocess_function=False)
test_dataset = CustomDataset(X_test, y_test, DIRECTORY_PATH, mode='test', include_preprocess_function=False)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)


    # Compute class weights using class indices
y_train_indices = np.argmax(y_train, axis=1)
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_indices), y=y_train_indices)
class_weights = torch.tensor(class_weights, dtype=torch.float32)

epochs_vgg16 = 3
learning_rate_vgg = 0.0001
model_vgg = NetworkVGG()
loss_function = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model_vgg.parameters(), lr=learning_rate_vgg)
train_losses = []
val_losses = []
f1_scores_train = []
f1_scores_val = []
reports = []

In [119]:
epoch = 10

for t in range(epoch):
    print(f"Epoch {t+31}\n-------------------------------")
    epoch_train_loss, f1_score_train = train(train_dataloader, model_vgg, loss_function, optimizer)
    epoch_val_loss, f1_score_val, report = validation(validation_dataloader, model_vgg, loss_function)
    train_losses.append(epoch_train_loss)
    val_losses.append(epoch_val_loss)
    f1_scores_val.append(f1_score_val)
    f1_scores_train.append(f1_score_train)
    save_model(train_dataloader, validation_dataloader, test_dataloader, optimizer, model_vgg, train_losses, val_losses, epoch=t+30)
print("Done!")

Epoch 31
-------------------------------


: 